In [ ]:
# ============================================================
# KMU Comovement — 공행성 필터 + 견고화 베이스라인 (Colab-ready)
# ============================================================

# 0) Install & Imports
!pip -q install pandas numpy lightgbm statsmodels fastdtw

import os, gc, warnings, math
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from typing import Tuple, Optional
from collections import defaultdict

from scipy.stats import pearsonr
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from statsmodels.tsa.stattools import grangercausalitytests

from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMRegressor, early_stopping, log_evaluation

# 1) Config
DATA_DIR = Path(".")
TRAIN_PATH = DATA_DIR / "train.csv"
SUB_PATH   = DATA_DIR / "sample_submission.csv"
OUT_PATH   = DATA_DIR / "submission.csv"

SEED = 42
np.random.seed(SEED)

MAX_LAG       = 6
USE_GRANGER   = False      # Granger 쓸지 여부 (True로 바꿔도 됨, 느려질 수 있음)
N_SPLITS      = 5

# 공행성 판정 관련 하이퍼파라미터
COMOV_THRESHOLD = 0.20     # 공행성 점수(상관계수 기반)가 이 값 이상일 때만 제출
MIN_OBS         = 18       # A,B 공통 관측 최소 개수
MIN_MEAN_VAL    = 10.0     # B의 월별 평균 value가 이 값 이하면 노이즈로 버림
CAP_QUANTILE    = 0.98     # 예측값 상단 캡: 과거 value의 상위 2% 지점

LGB_PARAMS = dict(
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=63,
    max_depth=8,
    subsample=0.9,
    colsample_bytree=0.9,
    min_data_in_leaf=25,
    min_sum_hessian_in_leaf=1.0,
    reg_lambda=1.0,
    reg_alpha=0.1,
    objective="rmse",
    random_state=SEED,
    verbosity=-1,
    force_row_wise=True
)

# 2) Load
train = pd.read_csv(TRAIN_PATH)
sub   = pd.read_csv(SUB_PATH)

assert {"value","weight","quantity"}.issubset(train.columns), "train.csv에 value/weight/quantity가 없습니다."
assert {"leading_item_id","following_item_id"}.issubset(sub.columns), "submission 스키마 확인 필요"

# 날짜 생성: year + month -> 그 달의 1일
if {"year","month"}.issubset(train.columns):
    train["date"] = pd.to_datetime(
        dict(year=train["year"], month=train["month"], day=1),
        errors="coerce"
    )
elif "date" in train.columns:
    train["date"] = pd.to_datetime(train["date"], errors="coerce")
else:
    raise ValueError("연-월 혹은 date 컬럼 필요")

# 3) 월 집계 + 보조 피처
train["is_value_only"] = (train["value"]>0) & (train["weight"]==0) & (train["quantity"]==0)
train["has_physical"]  = (train["weight"]>0) | (train["quantity"]>0)

agg_map = {"value":"sum","weight":"sum","quantity":"sum","is_value_only":"sum"}
if "hs4" in train.columns: agg_map["hs4"] = "first"
if "type" in train.columns: agg_map["type"] = "first"

monthly = train.groupby(["item_id","date"], as_index=False).agg(agg_map)

# 추가 통계 피처
n_trades = train.groupby(["item_id","date"])["value"].size().rename("n_trades").reset_index()
sum_value_only = (
    train.loc[train["is_value_only"]==1]
         .groupby(["item_id","date"])["value"].sum()
         .rename("sum_value_only").reset_index()
)
mix = (
    train.groupby(["item_id","date"])
         .agg(any_physical=("has_physical","any"),
              any_value_only=("is_value_only","any"))
         .assign(has_mixed=lambda d: d["any_physical"] & d["any_value_only"])
         .reset_index()[["item_id","date","has_mixed"]]
)

monthly = (monthly
           .merge(n_trades, on=["item_id","date"], how="left")
           .merge(sum_value_only, on=["item_id","date"], how="left")
           .merge(mix, on=["item_id","date"], how="left"))

monthly["has_mixed"] = monthly["has_mixed"].fillna(False).astype(int)
monthly["sum_value_only"] = monthly["sum_value_only"].fillna(0.0)
monthly["vo_ratio"] = np.where(
    monthly["value"]>0,
    monthly["sum_value_only"]/monthly["value"],
    0.0
)

# 4) 패널 (date x item_id)
panel = monthly.pivot(index="date", columns="item_id", values="value").sort_index()
date_min, date_max = panel.index.min(), panel.index.max()
target_month = pd.to_datetime("2025-08-01")
assert date_max >= pd.to_datetime("2025-07-01"), "데이터 최종월이 2025-07 이전입니다. target_month를 재확인하세요."

# 5) 공행성 유틸
def ccf_best_lag(a: pd.Series, b: pd.Series, max_lag=6, min_obs=18):
    """
    A: 선행 후보, B: 후행 후보
    공행성은 '무역량 변동' 기준 → 퍼센트 변화 시계열 기준으로 상관 계산
    """
    # 1) 무역량 변동 시계열로 변환 (퍼센트 변화)
    a_chg = a.pct_change().replace([np.inf, -np.inf], np.nan)
    b_chg = b.pct_change().replace([np.inf, -np.inf], np.nan)

    best_lag, best_corr = None, None
    for lag in range(1, max_lag + 1):
        # A_t 와 B_{t+lag} 비교
        a_cut   = a_chg.iloc[:-lag]
        b_shift = b_chg.shift(-lag).iloc[:-lag]

        valid = a_cut.notna() & b_shift.notna()
        if valid.sum() < min_obs:
            continue

        try:
            r, _ = pearsonr(a_cut[valid], b_shift[valid])
        except Exception:
            continue

        if (best_corr is None) or (r > best_corr):
            best_corr, best_lag = r, lag

    return best_lag, best_corr


def granger_min_p(a: pd.Series, b: pd.Series, maxlag=6, min_obs=36):
    df = pd.DataFrame({"B": b, "A": a}).dropna()
    if len(df) < max(min_obs, maxlag*3):
        return None
    try:
        res = grangercausalitytests(df[["B","A"]], maxlag=maxlag, verbose=False)
    except Exception:
        return None
    pvals = [out[0]["ssr_ftest"][1] for _, out in res.items()]
    return float(np.nanmin(pvals)) if pvals else None


def pick_lead_for_B(B: str, candidates_A: list, panel_df: pd.DataFrame, max_lag=6):
    """
    B에 대해, candidates_A 중에서 가장 공행성이 강한 리드 A를 찾는다.
    리턴: (A, lag, score) 또는 None
    score 기본값은 상관계수, USE_GRANGER True면 corr + Granger 조합.
    """
    if B not in panel_df.columns:
        return None
    b = panel_df[B]
    best = None

    for A in candidates_A:
        if A == B or A not in panel_df.columns:
            continue
        a = panel_df[A]
        lag, corr = ccf_best_lag(a, b, max_lag=max_lag, min_obs=MIN_OBS)
        if lag is None or corr is None:
            continue
        score = corr
        if USE_GRANGER:
            gp = granger_min_p(a, b, maxlag=max_lag)
            if gp is not None:
                score = 0.7 * corr + 0.3 * (1 - gp)
        if (best is None) or (score > best[-1]):
            best = (A, lag, score)

    return best


# 공행성 필터 함수
def is_comoving_pair(A: str, B: str, score: float,
                     panel_df: pd.DataFrame,
                     monthly_df: pd.DataFrame,
                     threshold: float) -> bool:
    """
    (A,B) 쌍이 공행성 있다고 볼지 여부를 판단.
    - score >= threshold
    - 공통 관측 수 >= MIN_OBS
    - B의 평균 value >= MIN_MEAN_VAL
    """
    if score < threshold:
        return False

    if (A not in panel_df.columns) or (B not in panel_df.columns):
        return False

    ab = panel_df[[A, B]].dropna()
    if len(ab) < MIN_OBS:
        return False

    mean_val_B = monthly_df.loc[monthly_df["item_id"]==B, "value"].mean()
    if np.isnan(mean_val_B) or (mean_val_B < MIN_MEAN_VAL):
        return False

    return True

# 6) 피처 생성
AUX_COLS = ["n_trades","has_mixed","vo_ratio","weight","quantity","sum_value_only"]

def build_features(monthly_df, panel_df, A, B, lag, max_own_lag=6, horizon=1):
    btab = (monthly_df[monthly_df["item_id"]==B]
            .set_index("date").sort_index())
    feat = pd.DataFrame(index=panel_df.index)

    b_series = panel_df.get(B)
    if b_series is None:
        return None, None

    # 자기 시차
    for l in range(1, max_own_lag+1):
        feat[f"B_lag{l}"] = b_series.shift(l)

    # A → B 리드
    if A is not None and A in panel_df.columns and lag is not None:
        feat[f"A({A})_lag{lag}"] = panel_df[A].shift(lag)

    # 보조 피처
    for c in AUX_COLS:
        if c not in btab.columns:
            btab[c] = 0.0
    feat = feat.join(btab[AUX_COLS], how="left")

    # 스케일 안정화: log1p + 음수 제거(lower=0 사용)
    for c in list(feat.columns):
        feat[c] = np.log1p(
            pd.to_numeric(feat[c], errors="coerce").clip(lower=0)
        ).replace([np.inf, -np.inf], 0)

    # 타깃 (한 달 뒤)
    y = b_series.shift(-horizon)
    y = np.log1p(
        pd.to_numeric(y, errors="coerce").clip(lower=0)
    ).replace([np.inf, -np.inf], 0)

    valid_idx = feat.dropna(how="all").index.intersection(y.dropna().index)
    X = feat.loc[valid_idx].copy()

    # 상수열 제거
    nunique = X.nunique(dropna=True)
    drop_const = nunique[nunique <= 1].index.tolist()
    if drop_const:
        X.drop(columns=drop_const, inplace=True, errors="ignore")

    X = X.fillna(0.0)
    y = y.loc[X.index]

    return X, y

# 7) 학습 데이터 구축
pairs = sub[["leading_item_id","following_item_id"]].drop_duplicates().values.tolist()
B_to_As = defaultdict(list)
for A, B in pairs:
    B_to_As[B].append(A)

# 각 B에 대해 "가장 괜찮은 A" 선택 (훈련용)
best_map = {}
for B, A_list in B_to_As.items():
    sel = pick_lead_for_B(B, A_list, panel, max_lag=MAX_LAG)
    if sel is None:
        best_map[B] = None
    else:
        A_sel, lag_sel, score_sel = sel
        best_map[B] = (A_sel, lag_sel)

X_all_list, y_all_list, row_keys = [], [], []
for B, sel in best_map.items():
    if sel is None:
        # 리드가 없으면 자기 자신의 과거만 사용 (A=B, lag=1, 나중에 A 피처 삭제)
        A, lag = B, 1
        Xb, yb = build_features(
            monthly, panel,
            A=A, B=B, lag=lag,
            max_own_lag=MAX_LAG, horizon=1
        )
        if Xb is not None:
            Xb = Xb[[c for c in Xb.columns if not c.startswith("A(")]]
    else:
        A, lag = sel
        Xb, yb = build_features(
            monthly, panel,
            A=A, B=B, lag=lag,
            max_own_lag=MAX_LAG, horizon=1
        )

    if Xb is None or yb is None:
        continue
    if len(Xb) >= 10:
        X_all_list.append(Xb)
        y_all_list.append(yb)
        row_keys += [(B, idx) for idx in Xb.index]

if not X_all_list:
    raise RuntimeError("유효 학습 샘플이 없습니다. (데이터/기간/결측 확인)")

# 공통 컬럼만 사용
common_cols = set(X_all_list[0].columns)
for Xi in X_all_list[1:]:
    common_cols &= set(Xi.columns)
common_cols = sorted(list(common_cols))
X_all_list = [Xi[common_cols].copy() for Xi in X_all_list]

X_all = pd.concat(X_all_list, axis=0).fillna(0.0)
y_all = pd.concat(y_all_list, axis=0)

# 타깃 분산 확인
if y_all.std() < 1e-6:
    USE_NAIVE_ONLY = True
else:
    USE_NAIVE_ONLY = False

# 8) 시계열 CV + 학습
if not USE_NAIVE_ONLY:
    tscv = TimeSeriesSplit(n_splits=N_SPLITS)
    model = LGBMRegressor(**LGB_PARAMS)
    for tr_idx, va_idx in tscv.split(X_all):
        model = LGBMRegressor(**LGB_PARAMS)
        model.fit(
            X_all.iloc[tr_idx], y_all.iloc[tr_idx],
            eval_set=[(X_all.iloc[va_idx], y_all.iloc[va_idx])],
            eval_metric="rmse",
            callbacks=[
                early_stopping(stopping_rounds=200, verbose=False),
                log_evaluation(period=0)
            ],
        )
    # 전체 재학습
    model = LGBMRegressor(**LGB_PARAMS)
    model.fit(X_all, y_all, callbacks=[log_evaluation(period=0)])
else:
    model = None

# 9) 2025-08 예측 (공행성 있는 쌍만 제출)
def predict_one_pair(A: Optional[str], B: str, lag: Optional[int]) -> float:
    idx = pd.to_datetime("2025-07-01")
    if idx not in panel.index:
        idx = panel.index.max()

    feat = {}

    # B 자기 시차
    for l in range(1, MAX_LAG+1):
        val = panel[B].shift(l).reindex([idx]).iloc[0] if B in panel.columns else np.nan
        feat[f"B_lag{l}"] = val

    # A 리드
    if A is not None and A in panel.columns and lag is not None:
        feat[f"A({A})_lag{lag}"] = panel[A].shift(lag).reindex([idx]).iloc[0]

    # 보조 피처
    btab = (monthly[monthly["item_id"]==B]
            .set_index("date").sort_index())
    for c in AUX_COLS:
        if (c in btab.columns) and (idx in btab.index):
            feat[c] = btab[c].reindex([idx]).iloc[0]
        else:
            feat[c] = 0.0

    Xp = pd.DataFrame([feat])
    for c in Xp.columns:
        Xp[c] = np.log1p(
            pd.to_numeric(Xp[c], errors="coerce").clip(lower=0)
        ).replace([np.inf, -np.inf], 0)

    if not USE_NAIVE_ONLY:
        for c in common_cols:
            if c not in Xp.columns:
                Xp[c] = 0.0
        Xp = Xp[common_cols].fillna(0.0)

    # 예측
    if USE_NAIVE_ONLY or model is None:
        yhat_lin = float(panel[B].reindex([idx]).iloc[0])
        return yhat_lin

    try:
        yhat_log = float(model.predict(Xp)[0])
        yhat_lin = np.expm1(yhat_log)
        if not np.isfinite(yhat_lin):
            raise ValueError
    except Exception:
        yhat_lin = float(panel[B].reindex([idx]).iloc[0])

    return yhat_lin


# === 공행성 있는 쌍만 모아서 submission 생성 ===
pred_rows = []

for _, row in sub.iterrows():
    A = row["leading_item_id"]
    B = row["following_item_id"]

    picked = pick_lead_for_B(B, [A], panel, max_lag=MAX_LAG)
    if picked is None:
        continue

    A_star, lag_star, score = picked

    # 공행성 필터 통과 못하면 제출 X
    if not is_comoving_pair(A_star, B, score, panel, monthly, COMOV_THRESHOLD):
        continue

    # 무역량 예측
    yhat = predict_one_pair(A_star, B, lag_star)

    # 과거 분포 기반 상단 캡
    hist = panel[B].dropna()
    if len(hist) >= 10:
        cap = float(hist.quantile(CAP_QUANTILE))
        if np.isfinite(cap) and cap > 0:
            yhat = min(yhat, cap)

    yhat = max(yhat, 0)
    yhat = int(round(yhat))

    pred_rows.append({
        "leading_item_id": A,
        "following_item_id": B,
        "value": yhat,
    })

sub_out = pd.DataFrame(pred_rows,
                       columns=["leading_item_id","following_item_id","value"])
sub_out.to_csv(OUT_PATH, index=False, encoding="utf-8")
print(f"Saved: {OUT_PATH}, shape={sub_out.shape}")


Saved: submission.csv, shape=(4455, 3)
